In [2]:
# Load packages
import tensorflow as tf
import pandas as pd
from tensorflow import keras
import numpy as np
import pandas as pd
import os
import scipy as scp
import scipy.stats as scps
import time
from datetime import datetime


# Load my own functions
import dnnregressor_train_eval_keras as dnnk
from kde_training_utilities import kde_load_data
from kde_training_utilities import kde_make_train_test_split
import make_data_wfpt as mdw

In [3]:
# Handle some cuda business

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="3"

from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 9048756146472092664
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 1286736053566926672
physical_device_desc: "device: XLA_CPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 12048649421
locality {
  bus_id: 2
  numa_node: 1
  links {
  }
}
incarnation: 10892668698647814911
physical_device_desc: "device: 0, name: GeForce GTX TITAN X, pci bus id: 0000:83:00.0, compute capability: 5.2"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 9910625608849971321
physical_device_desc: "device: XLA_GPU device"
]


In [4]:
# Make dnnk class (cpm for choice probability model)
cpm = dnnk.dnn_trainer()

# Define folder in which dataset lies
data_folder = '/media/data_cifs/afengler/data/kde/ornstein_uhlenbeck/train_test_data/'

In [5]:
# Make train test split
kde_make_train_test_split(folder = data_folder,
                          p_train = 0.8)

get files in folder
check if we have a train and test sets already
folder clean so proceeding...
read, concatenate and shuffle data
get training and test indices
writing to file...


'success'

In [6]:
# Load train test split
cpm.data['train_features'], cpm.data['train_labels'], cpm.data['test_features'], cpm.data['test_labels'] = kde_load_data(folder = data_folder)

In [ ]:
cpm.data['test_features'].shape

In [ ]:
cpm.data['train_features']

In [ ]:
cpm.data['train_features'].iloc[171247010, :]

In [ ]:
cpm.data['train_features']['log_l'] = cpm.data['train_labels']

In [ ]:
cpm.data['train_features'].sort_values(by = 'log_l')

In [ ]:
cpm.data['train_features']

In [ ]:
cpm.data['train_features'].iloc[22428, :]

In [ ]:
cpm.data['train_labels'][22428, ]

In [7]:
# Make all parameters we can specify explicit
# Model parameters
cpm.model_params

{'input_shape': 3,
 'output_shape': 1,
 'output_activation': 'sigmoid',
 'hidden_layers': [20, 20, 20, 20],
 'hidden_activations': ['relu', 'relu', 'relu', 'relu'],
 'l1_activation': [0.0, 0.0, 0.0, 0.0],
 'l2_activation': [0.0, 0.0, 0.0, 0.0],
 'l1_kernel': [0.0, 0.0, 0.0, 0.0],
 'l2_kernel': [0.0, 0.0, 0.0, 0.0],
 'optimizer': 'Nadam',
 'loss': 'mse',
 'metrics': ['mse']}

In [8]:
# Parameters governing training
cpm.train_params

{'callback_funs': ['ReduceLROnPlateau', 'EarlyStopping', 'ModelCheckpoint'],
 'plateau_patience': 10,
 'min_delta': 0.0001,
 'early_stopping_patience': 15,
 'callback_monitor': 'loss',
 'min_learning_rate': 1e-07,
 'red_coef_learning_rate': 0.1,
 'ckpt_period': 10,
 'ckpt_save_best_only': True,
 'ckpt_save_weights_only': True,
 'max_train_epochs': 2000,
 'batch_size': 10000,
 'warm_start': False,
 'checkpoint': 'ckpt',
 'model_cnt': 0}

In [9]:
# Parameters concerning data storage
cpm.data_params

{'data_type': 'choice_probabilities',
 'model_directory': '/media/data_cifs/afengler/git_repos/nn_likelihoods/keras_models',
 'checkpoint': 'ckpt',
 'model_name': 'dnnregressor',
 'data_type_signature': '_choice_probabilities_analytic_',
 'timestamp': '06_27_19_22_30_06',
 'training_data_size': 2500000}

In [11]:
# If necessary, specify new set of parameters here:
# Model params
cpm.model_params['output_activation'] = 'linear'
cpm.model_params['hidden_layers'] = [20, 40, 60, 80, 100, 120]
cpm.model_params['hidden_activations'] = ['relu', 'relu', 'relu', 'relu', 'relu', 'relu']
cpm.model_params['input_shape'] = cpm.data['train_features'].shape[1]
# cpm.model_params['l1_activation'] = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
# cpm.model_params['l2_activation'] = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
cpm.model_params['l1_kernel'] = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
cpm.model_params['l2_kernel'] = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
# Train params
cpm.train_params['batch_size'] = 1000000
cpm.train_params['max_train_epochs'] = 250
cpm.train_params['min_delta'] = 0.00001


# Data params
cpm.data_params['data_type'] = 'kde'
cpm.data_params['data_type_signature'] = '_ornstein_uhlenbeck_'
cpm.data_params['training_data_size'] = cpm.data['train_features'].shape[0]
cpm.data_params['timestamp'] = datetime.now().strftime('%m_%d_%y_%H_%M_%S')
cpm.data_params['model_directory'] = '/media/data_cifs/afengler/data/kde/linear_collapse/keras_models/'

In [12]:
# Make model
cpm.keras_model_generate(save_model = True)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


In [ ]:
# Train model
cpm.run_training(save_history = True, 
                 warm_start = False)

Train on 191409589 samples, validate on 47844971 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/250
191409589/191409589 [==============================] - 334s 2us/sample - loss: 0.1637 - mean_squared_error: 0.1637 - val_loss: 0.0945 - val_mean_squared_error: 0.0945
Epoch 2/250
191409589/191409589 [==============================] - 554s 3us/sample - loss: 0.0386 - mean_squared_error: 0.0386 - val_loss: 0.0230 - val_mean_squared_error: 0.0230
Epoch 3/250
191409589/191409589 [==============================] - 567s 3us/sample - loss: 0.0243 - mean_squared_error: 0.0243 - val_loss: 0.0337 - val_mean_squared_error: 0.0337
Epoch 4/250
191409589/191409589 [==============================] - 580s 3us/sample - loss: 0.0178 - mean_squared_error: 0.0178 - val_loss: 0.0232 - val_mean_squared_error: 0.0232
Epoch 5/250
191409589/191409589 [==============================] - 578s 3us/sample - loss: 0.0143 - mean_squared_error: 0.0143 - val_loss: 0.0122 - val_mean_squared_error: 0.0122


191409589/191409589 [==============================] - 568s 3us/sample - loss: 0.0037 - mean_squared_error: 0.0037 - val_loss: 0.0027 - val_mean_squared_error: 0.0027
Epoch 35/250
191409589/191409589 [==============================] - 565s 3us/sample - loss: 0.0034 - mean_squared_error: 0.0034 - val_loss: 0.0027 - val_mean_squared_error: 0.0027
Epoch 36/250
191409589/191409589 [==============================] - 561s 3us/sample - loss: 0.0035 - mean_squared_error: 0.0035 - val_loss: 0.0049 - val_mean_squared_error: 0.0049
Epoch 37/250
191409589/191409589 [==============================] - 571s 3us/sample - loss: 0.0034 - mean_squared_error: 0.0034 - val_loss: 0.0029 - val_mean_squared_error: 0.0029
Epoch 38/250
191409589/191409589 [==============================] - 569s 3us/sample - loss: 0.0033 - mean_squared_error: 0.0033 - val_loss: 0.0028 - val_mean_squared_error: 0.0028
Epoch 39/250
191409589/191409589 [==============================] - 570s 3us/sample - loss: 0.0032 - mean_squared